In [85]:
import numpy as np

class KNearestNeighbors():
    def __init__(self, X_train, y_train, n_neighbors=40, weights = 'uniform'):
    
        self.X_train = X_train
        self.y_train = y_train
        
        self.n_neighbors = n_neighbors
        self.weights = weights
        
        self.class_num = 3

        
    def euclidian_distance(self, a, b):
            
        return np.sqrt(np.sum((a-b)**2, axis=1))
    
    
    
    def kneighbors(self, X_test, return_distance=False):
       
        dist = []
        neigh_ind = []
        
        point_dist = [self.euclidian_distance(x_test, self.X_train) for x_test in X_test]

        for row in point_dist:
            enum_neigh = enumerate(row)
            sorted_neigh = sorted(enum_neigh, key=lambda x: x[1])[:self.n_neighbors]
    
            ind_list = [tup[0] for tup in sorted_neigh]
            dist_list = [tup[1] for tup in sorted_neigh]
    
            dist.append(dist_list)
            neigh_ind.append(ind_list)
        
        if return_distance:
            return np.array(dist), np.array(neigh_ind)
        
        return np.array(neigh_ind)
         
    
    def predict(self, X_test):
        
        if self.weights=='uniform':
            neighbors = self.kneighbors(X_test)
            y_pred = np.array([np.argmax(np.bincount(self.y_train[neighbor])) for neighbor in neighbors])
        
            return y_pred
    
        if self.weights=='distance':
        
            dist, neigh_ind = self.kneighbors(X_test, return_distance=True)
        
            inv_dist = 1/dist
            
            mean_inv_dist = inv_dist / np.sum(inv_dist, axis=1)[:, np.newaxis]
            
            proba = []
            
            for i, row in enumerate(mean_inv_dist):
                
                row_pred = self.y_train[neigh_ind[i]]
                
                for k in range(self.class_num):
                    indices = np.where(row_pred==k)
                    prob_ind = np.sum(row[indices])
                    proba.append(np.array(prob_ind))
        
            predict_proba = np.array(proba).reshape(X_test.shape[0], self.class_num)
            
            y_pred = np.array([np.argmax(item) for item in predict_proba])
            
            return y_pred
            
    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        
        return float(sum(y_pred == y_test))/ float(len(y_test))


In [79]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples = 1000, n_features=2, n_redundant=0, n_informative=2,
                             n_clusters_per_class=1, n_classes=3, random_state=21)


mu = np.mean(X, 0)
sigma = np.std(X, 0)

X = (X - mu ) / sigma

In [110]:
from sklearn.datasets import load_iris

dataset = load_iris()
X = dataset.data
y = dataset.target

In [111]:
data = np.hstack((X, y[:, np.newaxis]))
        
np.random.shuffle(data)

split_rate = 0.7

train, test = np.split(data, [int(split_rate*(data.shape[0]))])

X_train = train[:,:-1]
y_train = train[:, -1]

X_test = test[:,:-1]
y_test = test[:, -1]

y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [112]:
knn = KNearestNeighbors(X_train, y_train, weights='uniform')

In [113]:
knn.kneighbors(X_test)

array([[104,  82,  69, ...,  34,  57,  42],
       [ 47,  37,  63, ...,  67,   0,  79],
       [ 10,  60,  42, ...,  66,  50,  14],
       ...,
       [ 51, 104,  25, ...,  85,  64,  80],
       [ 41,  28,   2, ...,  67,   0,  68],
       [ 58, 100,  74, ...,  61,  65,  19]])

In [114]:
predicted_y = knn.predict(X_test)

In [115]:
predicted_y

array([1, 0, 1, 2, 1, 1, 1, 2, 0, 1, 1, 1, 2, 1, 2, 1, 2, 2, 0, 0, 0, 2,
       1, 1, 1, 1, 2, 1, 0, 0, 0, 0, 2, 0, 0, 1, 1, 2, 2, 2, 0, 1, 2, 0,
       1])

In [116]:
knn.score(X_test, y_test)

0.9111111111111111

In [86]:
from sklearn.datasets import load_iris

In [93]:
dataset = load_iris()

In [94]:
X = dataset.data

In [96]:
y = dataset.target

(150,)